In [ ]:
import warnings
warnings.filterwarnings("ignore")
import wrangle as w

import pandas as pd
import numpy as np

#splits
from sklearn.model_selection import train_test_split

#visualization
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import plotly.express as px

#scaling
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import precision_score, accuracy_score, recall_score, classification_report

#model
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression


# Acquire

In [ ]:
df = w.left_join_csv('austin_animal_outcomes.csv', 'austin_animal_intakes.csv', 'merged_data.csv')


# Prepare

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sqlalchemy import text, create_engine

import warnings
warnings.filterwarnings("ignore")

np.random.seed(42)

####################### Imports ############################

                                                        ############### Acquire Functions ###########################

def get_aa_data(fn, query, url):
    """
    check if file exists in my local directory, if not, pull from sql db
    return dataframe
    """
    if os.path.isfile(fn):
        print('csv file found and loaded')
        return pd.read_csv(fn, index_col=0)
    else:
        print('creating df and exporting csv')
        df = pd.read_sql(query, url)
        df.to_csv(fn)
        return df
    
def get_prep_aa(df):
    # made all column names lower case
    df.columns = df.columns.str.lower()
    df = df.apply(lambda x: x.astype(str).str.lower())
    new_columns = {
        'datetime_x': 'outcome_datetime',
        'datetime_y': 'intake_datetime',
        'monthyear_x': 'outcome_monthyear',
        'monthyear_y': 'intake_monthyear',
        'name_y': 'name',
        'breed_y': 'breed',
        'animal type_y': 'species',
        'outcome type': 'outcome',
        'color_y': 'color',
        'sex upon outcome': 'outcome_sex',
        'sex upon intake': 'intake_sex',
        'intake type': 'intake_type',
        'age upon intake': 'intake_age',
        'age upon outcome': 'outcome_age',
        'date of birth': 'dob',
        'intake condition': 'intake_condition',
        'found location': 'found_location',
        'animal id': 'id'      
    }
    df = df.rename(columns=new_columns)
    #dropped unnecessary column names, outcome subtype, due to having over 119k of 193k rows empty, intake_monthyear, outcome_month_year, animal type_x, are predominantly the same, 
    columns_to_drop = ['outcome subtype', 'name_x', 'breed_x', 'animal type_x', 'color_x', 'intake_monthyear', 'outcome_monthyear']
    df = df.drop(columns=columns_to_drop)

    # dropping nulls
    df.dropna(subset=['intake_sex'], inplace=True)
    df.dropna(subset=['outcome'], inplace=True)

    # create dates
    df['outcome_date'] = pd.to_datetime(df['outcome_datetime']).dt.strftime('%m/%d/%Y').astype("datetime64")
    df['intake_date'] = pd.to_datetime(df['intake_datetime']).dt.strftime('%m/%d/%Y').astype("datetime64")
    df['dob'] = pd.to_datetime(df['dob'], format='%m/%d/%Y')

    # create ages
    df['intake_age'] = (df.intake_date - df.dob).dt.days
    df['outcome_age'] = (df.outcome_date - df.dob).dt.days

    # days in center
    df["tenure_days"] = (df['outcome_age'] - df['intake_age'] )
    # filter weird dates
    df = df[df.tenure_days > 0]

    # color and intake condition columns
    df = transform_color(df)
    df = transform_intake_condition(df)

    #filtered for cats and dogs
    df = df[df['species'].isin(['cat', 'dog'])]
    df = df[df['outcome'].isin(['adoption', 'transfer', 'rto-adopt', 'return to owner', 'euthanasia'])]
    df = df[df['intake_type'].isin(['stray', 'owner surrender', 'public assist', 'abandoned'])]

    # mix breeds columns
    df['mix_breeds'] = np.where(df['breed'].str.contains('mix', case=False, na=False), 1, 0)
    df['two_breeds'] = np.where(df['breed'].str.contains('/', case=False, na=False), 1, 0)
    df['pure_breed'] = np.where(df['breed'].isin(['/', 'mix']), 1, 0)

    # if pet has a name 1, if not 0 place in column has_name
    df['has_name'] = np.where(df['name'] != 'nan', 1, 0)

    # dropping unknown sex from df
    df = df[(df.intake_sex != 'unknown') & (df.intake_sex != 'nan')]

    # keep these columns
    keep_col= ['has_name', 'outcome', 'dob',
               'species', 'intake_type', 'intake_condition',
               'intake_date', 'outcome_date', 'intake_age',
               'outcome_age', 'tenure_days', 'intake_sex',
               'breed', 'mix_breeds', 'two_breeds', 'pure_breed',
               'primary_color', 'is_tabby', 'mix_color']
    df = df[keep_col]

    dummies_df = pd.get_dummies(df, columns=['outcome', 'species', 'intake_type',
                                             'intake_condition', 'intake_sex', 'primary_color'])
    model_df = dummies_df.drop(columns=['dob', 'intake_date', 'outcome_date', 'breed'])
    return df, model_df

                                #################### Prepare Functions ##########################

def transform_intake_condition(df):
    """
    Transforms the intake_condition column of a DataFrame by performing several operations.

    Args:
        df (pandas.DataFrame): The input DataFrame containing an 'intake_condition' column.

    Returns:
        pandas.DataFrame: The transformed DataFrame.

    """

    df = df.apply(lambda x: x.astype(str).str.lower())

    # Change 'Feral', 'Neurologic', 'Behavior', 'Space' to 'mental' category
    df['intake_condition'] = df['intake_condition'].replace(['feral', 'neurologic', 'behavior', 'space'], 'mental')

    # Set values indicating medical attention
    df['intake_condition'] = df['intake_condition'].replace(['nursing', 'neonatal', 'medical', 'pregnant', 'med attn', 
                                                            'med urgent', 'parvo', 'agonal', 'panleuk'], 'medical attention')

    # Drop rows with 'other', 'unknown', and 'nan' values
    df = df[df['intake_condition'].isin(['other', 'unknown', 'nan']) == False]

    return df

    
def transform_color(df):
    """
    Transforms the color column of a DataFrame by performing several operations.

    Args:
        df (pandas.DataFrame): The input DataFrame containing a 'color' column.

    Returns:
        pandas.DataFrame: The transformed DataFrame with additional columns.

    """

    # lowercase everything
    df = df.apply(lambda x: x.astype(str).str.lower())

    # Add spaces between color names separated by slashes
    df['color'] = df['color'].str.replace('/', ' / ')

    # Replace color names with their corresponding standard names
    replacements = {
        'chocolate': 'brown',
        'liver': 'brown',
        'ruddy': 'brown',
        'apricot': 'orange',
        'pink': 'red',
        'cream': 'white',
        'flame point': 'white',
        'blue': 'gray',
        'silver': 'gray',
        'yellow': 'gold',
        'torbie': 'tricolor',
        'tortie': 'tricolor',
        'calico': 'tricolor'
    }
    df['color'] = df['color'].replace(replacements, regex=True)

    # Create new column 'primary_color' with the first color
    colors = ['black', 'brown', 'white', 'tan', 'brindle', 'gray', 'fawn', 'red', 'sable', 'buff', 'orange', 'blue',
              'tricolor', 'gold', 'cream', 'lynx point', 'seal point', 'agouti', 'lilac point']
    for color in colors:
        df.loc[df['color'].str.startswith(color), 'primary_color'] = color

    # Drop rows with 'unknown' color
    df = df[df['color'] != 'unknown']

    # Create column indicating if the animal has a tabby pattern
    df['is_tabby'] = df['color'].str.contains('tabby').astype(int)

    # Create column indicating if the animal has mixed colors
    df["mix_color"] = np.where(df['color'].str.contains(r'\/|tricolor|torbie|tortie'), 1, 0)

    df = df.drop(columns=["color"])

    return df


def split_data(df, target_variable):
    '''
    Takes in two arguments the dataframe name and the ("target_variable" - must be in string format) to stratify  and 
    return train, validate, test subset dataframes will output train, validate, and test in that order.
    '''
    train, test = train_test_split(df, #first split
                                   test_size=.2, 
                                   random_state=123, 
                                   stratify= df[target_variable])
    train, validate = train_test_split(train, #second split
                                    test_size=.25, 
                                    random_state=123, 
                                    stratify=train[target_variable])
    return train, validate, test

In [ ]:
df, model_df = w.prep_df(df)

In [ ]:
df.head()

In [ ]:
df.outcome.value_counts()

In [ ]:
df.condition.value_counts()

In [ ]:
df.breed.value_counts()

In [ ]:
#df.color.head(10)

In [ ]:
model_df.columns

In [ ]:
train,validate,test = w.split_data(df, 'outcome')

In [ ]:
train.head()


# Question and Hypothesis Statement

Does Tenure have a correlation to adoption

In [ ]:
def run_tenure_ttest(data):
    '''
    runs a Ttest for Tenure county vs Outcome
    '''
    x = data['outcome_adoption']
    y = data['tenure_days']
    # Perform t-test
    t_statistic, p_value = stats.ttest_ind(x, y)
    # Decide whether to reject the null hypothesis
    alpha = 0.05
    if p_value >= alpha:
        decision = "Fail to Reject Null Hypothesis"
    else:
        decision = "Reject Null Hypothesis"
# Create a DataFrame to store the results
    results = pd.DataFrame({
        'T-Statistic': [t_statistic],
        'P-Value': [p_value],
        'Decision': [decision]})
    return results

In [ ]:
#model_df.tenure_days = model_df.tenure_days.astype('float')

In [ ]:
#train.tenure_days.info()


##### Hypothisis: County\n",
$H_0$ : Outcome is independent of tenure <br>\
$H_a$ : Outcome is not independent of tenure

In [ ]:
#import scipy.stats as stats

#run_tenure_ttest(model_df)

In [ ]:
#fig = px.box(model_df, x="outcome_adoption", y="tenure_days", notched=True)
#fig.show()

In [ ]:
#model_df.tenure_days.describe()

# Conclusion

Tenure wont work due to lack of data integrity

In [ ]:
train.outcome.value_counts()

In [ ]:
train.columns

# Question and Hypothesis Statement

Is there a corolation between intake_sex and outcome

##### Hypothisis: County\n",
$H_0$ : Outcome is independent of sex <br>\
$H_a$ : Outcome is not independent of sex

In [ ]:
import pandas as pd
import plotly.express as px

#df = train
#grouped_data = df.groupby(["sex", "outcome"]).size().reset_index(name="count")
#
#fig = px.bar(grouped_data, x="sex", y="count", color="outcome", barmode="group")
#fig.show()



In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# Create a contingency table
contingency_table = pd.crosstab(train['sex'], train['outcome'])

# Perform the chi-square test
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

# Print the results
print("Chi-square statistic:", chi2)
print("p-value:", p_value)


In [ ]:
contingency_table

# Conclusion

Ther is a relationship here

# Question and Hypothesis Statement

are taby cats more likely to get adopted than non taby cats

In [ ]:
cats = train[train.species == 'cat']

In [ ]:
cats.is_tabby.unique()

In [ ]:
cats.is_tabby.value_counts()

In [ ]:

#grouped_data = cats.groupby(["is_tabby", "outcome"]).size().reset_index(name="count")
#
#fig = px.bar(grouped_data, x="is_tabby", y="count", color="outcome", barmode="group")
#fig.show()

In [ ]:
# Create a contingency table
contingency_table_tabby = pd.crosstab(cats['is_tabby'], train['outcome'])

# Perform the chi-square test
chi2, p_value, dof, expected = chi2_contingency(contingency_table_tabby)

# Print the results
print("Chi-square statistic:", chi2)
print("p-value:", p_value)

In [ ]:
contingency_table_tabby

In [ ]:
6033/cats.is_tabby.value_counts()[0]

In [ ]:
902/cats.is_tabby.value_counts()[0]

In [ ]:
5298/cats.is_tabby.value_counts()[1]

In [ ]:
690/cats.is_tabby.value_counts()[1]

# Conclusion

the outcome for cats based on it being a tabby or not seems to propotionate despite CHI2 indicating some relationship

## Models

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def get_baseline(y_train):
    '''
    this function returns a baseline for accuracy
    '''
    baseline_prediction = y_train.mode()
    # Predict the majority class in the training set
    baseline_pred = [baseline_prediction] * len(y_train)
    accuracy = accuracy_score(y_train, baseline_pred)
    baseline_results = {'Baseline': [baseline_prediction],'Metric': ['Accuracy'], 'Score': [accuracy]}
    baseline_df = pd.DataFrame(data=baseline_results)
    return baseline_df 

In [ ]:
#creating X,y
def get_xy():
    '''
    This function generates X and y for train, validate, and test
    '''
    # Acquiring data
    df = w.left_join_csv('austin_animal_outcomes.csv', 'austin_animal_intakes.csv', 'merged_data.csv')
    # Running preperation 
    df, model_df = w.prep_df(df)
    # Split
    train, validate, test = w.split_data(model_df,'outcome')
    # create X & y version of train, where y is a series with just the target variable and X are all the features.    
    X_train = train.drop(['outcome'], axis=1)
    y_train = train.outcome
    X_validate = validate.drop(['outcome'], axis=1)
    y_validate = validate.outcome
    X_test = test.drop(['outcome'], axis=1)
    y_test = test.outcome
    return X_train,y_train,X_validate,y_validate,X_test,y_test

In [ ]:
X_train, y_train, X_validate, y_validate, X_test, y_test = get_xy()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, recall_score, precision_score
import pandas as pd


def create_models(seed=123):
    '''
    Create a list of machine learning models.
            Parameters:
                    seed (integer): random seed of the models
            Returns:
                    models (list): list containing the models
    This includes best fit hyperparamaenters                
    '''
    models = []
    models.append(('k_nearest_neighbors', KNeighborsClassifier(n_neighbors=100)))
    models.append(('logistic_regression', LogisticRegression(random_state=seed)))
    models.append(('DecisionTreeClassifier', DecisionTreeClassifier(max_depth=3,min_samples_split=4,random_state=seed)))
    models.append(('random_forest', RandomForestClassifier(max_depth=3,random_state=seed)))
    models.append(('support_vector_machine', SVC(random_state=seed)))
    models.append(('naive_bayes', GaussianNB()))
    models.append(('gradient_boosting', GradientBoostingClassifier(random_state=seed)))
    return models

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score


def get_models():
    # create models list
    models = create_models(seed=123)
    X_train, y_train, X_validate, y_validate, X_test, y_test = get_xy()
    # initialize results dataframe
    results = pd.DataFrame(columns=['model', 'set', 'accuracy', 'recall'])
    
    # loop through models and fit/predict on train and validate sets
    for name, model in models:
        # fit the model with the training data
        model.fit(X_train, y_train)
        
        # make predictions with the training data
        train_predictions = model.predict(X_train)
        
        # calculate training accuracy, recall, and precision
        train_accuracy = accuracy_score(y_train, train_predictions)
        train_recall = recall_score(y_train, train_predictions, average='weighted')
        train_precision = precision_score(y_train, train_predictions, average='weighted')
        
        # make predictions with the validation data
        val_predictions = model.predict(X_validate)
        
        # calculate validation accuracy, recall, and precision
        val_accuracy = accuracy_score(y_validate, val_predictions)
        val_recall = recall_score(y_validate, val_predictions, average='weighted')
        val_precision = precision_score(y_validate, val_predictions, average='weighted')

        
        # append results to dataframe
        results = results.append({'model': name, 'set': 'train', 'accuracy': train_accuracy, 'recall': train_recall, 'precision' : train_precision},ignore_index=True)
        results = results.append({'model': name, 'set': 'validate', 'accuracy': val_accuracy, 'recall': val_recall, 'precision' : val_precision}, ignore_index=True)
  

        '''
        this section left in case I want to return to printed format rather than data frame
        # print classifier accuracy and recall
        print('Classifier: {}, Train Accuracy: {}, Train Recall: {}, Validation Accuracy: {}, Validation Recall: {}'.format(name, train_accuracy, train_recall, val_accuracy, val_recall))
        '''
    return results

In [ ]:
get_models()

In [ ]:
get_baseline(y_train)